# DEMO:Manejo de Credenciales verificables centralizado y descentralizado

Acontinuacion realizaremos una demostracion que ilustra el manejo de credenciales verificables uno centralizado y otro descentralizado.Utilizamos criptografia de clave publica para firmar y verificar estas credenciales.

Simularemos la emisión, verificación, falsificación y revocación (en el caso centralizado) de credenciales digitales.

### Importacion de librerias a trabajar: Importacion de modulos y librerias





In [ ]:
import json  # para manejar JSON
import uuid  # para generar identificadores únicos
import time  # para obtener el tiempo actual
from typing import Dict, Any, Tuple, Set, Optional # para definir tipos

# para crear widgets interactivos ,para mostrar y limpiar salidas
import ipywidgets as widgets 
from IPython.display import display, clear_output 

# para manejar criptografia
from cryptography.hazmat.primitives import hashes, serialization 
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePrivateKey, EllipticCurvePublicKey
from cryptography.exceptions import InvalidSignature

#### Constantes

In [ ]:
# Se designan los nombre a los campos usados en los diccioarios que representan credenciales

CREDENTIAL_ID = "credential_id"
USER_ID = "user_id"
SUBJECT = "subject"
CLAIM = "claim"
VALUE = "value"
TIMESTAMP = "timestamp"
MAYOR_DE_EDAD_CLAIM = "mayor_de_edad"